In [1]:
using Pkg;
Pkg.add("Hwloc")

   Updating registry at `~/.julia/registries/General`


   Updating git-repo `https://github.com/JuliaRegistries/General.git`


Fetching: [========================================>]  100.0 %

  Resolving package versions...
  Installed Hwloc ─ v1.0.3
   Updating `~/Repos/advection-diffusion-catalysis/Project.toml`
  [0e44f5e4] + Hwloc v1.0.3
   Updating `~/Repos/advection-diffusion-catalysis/Manifest.toml`
  [0e44f5e4] + Hwloc v1.0.3
   Building Hwloc → `~/.julia/packages/Hwloc/1kB0k/deps/build.log`


In [2]:
using Hwloc
Hwloc.num_physical_cores()

┌ Info: Precompiling Hwloc [0e44f5e4-bd66-52a0-8798-143a42290a1d]
└ @ Base loading.jl:1260


20

In [1]:
using .Threads
nthreads()

10

In [ ]:
import Base.Threads.@spawn
using DrWatson
@quickactivate :Catalyst
using BenchmarkTools

function benchmark_distributed_evaluation(parameters,N)
    errors = zeros(N) # init model errors
    # we need to read the experiment data in this block
    input_exp = Float64[]
    output_exp = Float64[]
    for row in CSV.File(datadir("experiment/new-data.csv"); delim = " ")
        push!(input_exp, row.I)
        push!(output_exp, row.O)
    end
    # done reading 

    F = Array{Task}(undef,N)
     for i in 1:N
        F[i] = @spawn (Catalyst.solve(parameters[1], parameters[2], 1.,
                                      input_exp, output_exp, progress=false, 
                                      microcomp_type=:nonlinear,
                                      Q=parameters[3], kₙ=parameters[4],
                                      calibration=true))# save the model error
     end
        for i in 1:N
        errors[i] = fetch(F[i])
        end
    return errors
end

parameters = [1e-4, 1., 1., 10.]
@time benchmark_distributed_evaluation(parameters, 10)

In [18]:
import Base.Threads.@spawn 
using BenchmarkTools
function benchmark_threading_dummy(N)
errors = zeros(N)
F = Array{Task}(undef,N)
for i in 1:N
    F[i] = @spawn 10+10
end
for i in 1:N
    errors[i]= fetch(F[i])
end 
return errors
end 
@benchmark benchmark_threading_dummy(4)

BenchmarkTools.Trial: 
  memory estimate:  3.03 KiB
  allocs estimate:  22
  --------------
  minimum time:     6.250 μs (0.00% GC)
  median time:      15.495 μs (0.00% GC)
  mean time:        16.088 μs (0.00% GC)
  maximum time:     162.071 μs (0.00% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [2]:
Base.Threads.nthreads()


10

In [19]:
import Base.Threads.@spawn 
using BenchmarkTools
function benchmark_threading_dummy(N)
errors = zeros(N)
F = Array{Task}(undef,N)
for i in 1:N
    F[i] = @spawn 10+10
end
for i in 1:N
    errors[i]= fetch(F[i])
end 
return errors
end 
@benchmark benchmark_threading_dummy(10)

BenchmarkTools.Trial: 
  memory estimate:  7.34 KiB
  allocs estimate:  52
  --------------
  minimum time:     10.116 μs (0.00% GC)
  median time:      22.277 μs (0.00% GC)
  mean time:        23.483 μs (2.11% GC)
  maximum time:     5.055 ms (97.96% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [20]:
import Base.Threads.@spawn 
using BenchmarkTools
function benchmark_serial_dummy(N)
errors = zeros(N)
for i in 1:N
    errors[i]= 10+10
end 
return errors
end 
@benchmark benchmark_serial_dummy(4)

BenchmarkTools.Trial: 
  memory estimate:  112 bytes
  allocs estimate:  1
  --------------
  minimum time:     40.514 ns (0.00% GC)
  median time:      43.803 ns (0.00% GC)
  mean time:        52.652 ns (9.76% GC)
  maximum time:     2.359 μs (97.09% GC)
  --------------
  samples:          10000
  evals/sample:     990

In [23]:
import Base.Threads.@spawn 
using BenchmarkTools
function benchmark_serial_dummy(N)
errors = zeros(N)
for i in 1:N
    errors[i]= 10+10
end 
return errors
end 
@benchmark benchmark_serial_dummy(100)

BenchmarkTools.Trial: 
  memory estimate:  896 bytes
  allocs estimate:  1
  --------------
  minimum time:     147.982 ns (0.00% GC)
  median time:      186.101 ns (0.00% GC)
  mean time:        231.410 ns (14.36% GC)
  maximum time:     9.131 μs (93.55% GC)
  --------------
  samples:          10000
  evals/sample:     870

In [25]:
import Base.Threads.@spawn 
using BenchmarkTools
function benchmark_threading_dummy(N)
errors = zeros(N)
F = Array{Task}(undef,N)
for i in 1:N
    F[i] = @spawn 10+10
end
for i in 1:N
    errors[i]= fetch(F[i])
end 
return errors
end 
@benchmark benchmark_threading_dummy(100)

BenchmarkTools.Trial: 
  memory estimate:  72.06 KiB
  allocs estimate:  502
  --------------
  minimum time:     84.908 μs (0.00% GC)
  median time:      124.793 μs (0.00% GC)
  mean time:        131.006 μs (5.00% GC)
  maximum time:     6.882 ms (96.21% GC)
  --------------
  samples:          10000
  evals/sample:     1

In [32]:
using .Threads
using BenchmarkTools
function benchmark_serial_dummy(N)
errors = zeros(N)
@inbounds @threads  for i in 1:N
    errors[i]= 10+10
end 
return errors
end 
@benchmark benchmark_serial_dummy(100)

BenchmarkTools.Trial: 
  memory estimate:  8.05 KiB
  allocs estimate:  54
  --------------
  minimum time:     33.318 μs (0.00% GC)
  median time:      54.354 μs (0.00% GC)
  mean time:        53.803 μs (2.11% GC)
  maximum time:     11.607 ms (97.98% GC)
  --------------
  samples:          10000
  evals/sample:     1